In [1]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import pandas as pd
import torchvision.models as models 
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import matplotlib.pyplot as plt
import latent_plot as lp
from torch.autograd import Variable
from torch.optim import lr_scheduler
import seaborn as sns
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import copy
#from model import VAE

import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from data_loader import *
from model import convVAE
from sklearn.manifold import TSNE

In [2]:
#only on validation set
csv_path = '/home/vip/sayan-mandal/datasets/obj_criteria/20200619-objective.csv'
img_path = '/home/vip/sayan-mandal/datasets/obj_criteria/good_reduced/'

ResizeParam = 128
FD = FullDataLoader(csv_path, img_path,transform=transforms.Compose([transforms.Resize(ResizeParam), transforms.RandomHorizontalFlip(p=0.5) ,transforms.ToTensor()]))
FD2 = FullDataLoader(csv_path, img_path,transform=transforms.Compose([transforms.Resize(ResizeParam), transforms.ToTensor()]))

#random sampler
batch_size = 64
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(FD)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
val_indices =  indices[:split]

# Creating PT data samplers and loaders:
valid_sampler = SubsetRandomSampler(val_indices)

#train_loader = torch.utils.data.DataLoader(FD, batch_size=batch_size, sampler=train_sampler)
data_loader = torch.utils.data.DataLoader(FD2, batch_size=batch_size, sampler=valid_sampler)

In [3]:
#vae model structure
nl = 2048
model = convVAE((3,128,128),nl).cuda()
model.load_state_dict(torch.load('/home/vip/sayan-mandal/vae-convmodels/dfcvae_inp128_nl'+str(nl)+'_a1b1.torch'))
print(model)

convVAE(
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Conv2d(64, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Flatten(start_dim=1, end_dim=-1)
  )
  (fc1): Sequential(
    (0): Linear(in_features=16384, out_features=4096, bias=True)
    (1): BatchNo

In [4]:
for i, (data,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10) in enumerate(data_loader):
    y, _,_ =  model.encode(Variable(data).cuda())
    y = y.cpu().detach().numpy()
    if i == 0:
        yd = y
        lbl1,lbl2,lbl3,lbl4,lbl5,lbl6,lbl7,lbl8,lbl9,lbl10 = l1.numpy(),l2.numpy(),l3.numpy(),l4.numpy(),l5.numpy(),l6.numpy(),l7.numpy(),l8.numpy(),l9.numpy(),l10.numpy()
    else:
        yd = np.append(yd,y,axis=0)
        lbl1 = np.append(lbl1,l1.numpy(), axis=0)
        lbl2= np.append(lbl2,l2.numpy(), axis=0)
        lbl3= np.append(lbl3,l3.numpy(), axis=0)
        lbl4= np.append(lbl4,l4.numpy(), axis=0)
        lbl5= np.append(lbl5,l5.numpy(), axis=0)
        lbl6= np.append(lbl6,l6.numpy(), axis=0)
        lbl7= np.append(lbl7,l7.numpy(), axis=0)
        lbl8= np.append(lbl8,l8.numpy(), axis=0)
        lbl9= np.append(lbl9,l9.numpy(), axis=0)
        lbl10 = np.append(lbl10,l10.numpy(), axis=0)

In [7]:
yd.shape

(1725, 2048)

In [13]:
lbl1 = lbl1.reshape(-1,1)
lbl2 = lbl2.reshape(-1,1)
lbl3 = lbl3.reshape(-1,1)
lbl4 = lbl4.reshape(-1,1)
lbl5 = lbl5.reshape(-1,1)
lbl6 = lbl6.reshape(-1,1)
lbl7 = lbl7.reshape(-1,1)
lbl8 = lbl8.reshape(-1,1)
lbl9 = lbl9.reshape(-1,1)
lbl10 = lbl10.reshape(-1,1)

In [14]:
cols = ['class','mdprob','psdprob','md_inf','md_sup','rnfl_g','rnfl_ti','rnfl_ni','rnfl_ts','rnfl_ns']
for i in range(2048):
    cols += ['ls_'+str(i)]

In [22]:
to_df = np.concatenate((lbl1,lbl2,lbl3,lbl4,lbl5,lbl6,lbl7,lbl8,lbl9,lbl10,yd),axis=1)

In [23]:
df = pd.DataFrame(data=to_df, columns=cols)

In [24]:
df

,class,mdprob,psdprob,md_inf,md_sup,rnfl_g,rnfl_ti,rnfl_ni,rnfl_ts,rnfl_ns,...,ls_2038,ls_2039,ls_2040,ls_2041,ls_2042,ls_2043,ls_2044,ls_2045,ls_2046,ls_2047
0,1.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,2.0,1.0,...,-0.035497,0.017104,-0.024490,-0.003946,0.009751,-0.017343,-0.061130,0.020249,-0.026730,0.002883
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.059229,0.020277,-0.052141,-0.000271,-0.039300,0.032732,-0.040822,-0.031884,-0.044544,-0.066987
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.041434,0.092617,0.090972,-0.012806,-0.064488,0.037780,-0.033381,0.000841,-0.044127,0.032626
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.070935,-0.040561,-0.000830,-0.013262,-0.039083,-0.029831,-0.008518,-0.018122,-0.022468,-0.061820
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.033665,-0.055730,0.083801,0.018755,-0.152896,-0.028705,-0.033379,-0.043857,0.043309,0.038543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.019647,0.026008,-0.032711,0.002885,-0.044107,-0.026433,-0.000745,-0.043905,-0.048070,-0.023680
1721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.030018,-0.019068,-0.021037,0.007441,-0.001744,0.014481,0.006427,-0.040510,0.024882,0.046480
1722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.039180,0.028197,0.133708,-0.000668,0.091200,0.060185,0.023998,0.017122,0.026496,0.013872
1723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.033015,-0.010623,-0.032000,-0.003663,-0.067166,-0.012841,0.047152,-0.003716,-0.036523,-0.062790


In [25]:
df.to_csv('latentdat.csv',index=False)